<a href="https://colab.research.google.com/github/Lucasdz-7/Projeto-Integrador-2/blob/main/C%C3%B3digoAPi2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [14]:
import pandas as pd
import os
from google.colab import files
from google.colab import drive

In [22]:
drive.mount('/content/drive')
origem = '/content/drive/MyDrive/Data/Dados/'
DadosIbama = origem + 'dados_ibama.csv'

In [58]:
# Criando Dataframes

Dados_IBAMA = pd.read_csv(DadosIbama, low_memory=False, sep=';', encoding='utf-8')

In [ ]:
Dados_IBAMA.head(10)

In [59]:
# Filtrando para o período entre 2013 - 2023 e ordenando por ordem crescente

Dados_IBAMA_2013a2023 = Dados_IBAMA[(Dados_IBAMA["Ano"] >= 2013) & (Dados_IBAMA["Ano"] <= 2023)]
Dados_IBAMA_2013a2023 = Dados_IBAMA_2013a2023.sort_values(by='Ano', ascending=True)

In [26]:
Dados_IBAMA_2013a2023.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2941968 entries, 200354 to 281960
Data columns (total 16 columns):
 #   Column                   Dtype 
---  ------                   ----- 
 0   CNPJ                     object
 1   Razão Social             object
 2   Estado                   object
 3   Município                object
 4   Ano                      int64 
 5   Produto                  object
 6   Quantidade Transportada  object
 7   Unidade de Medida        object
 8   Tipo de Transporte       object
 9   Tipo de Armazenamento    object
 10  Plano de Emergência      object
 11  UF - origem              object
 12  Município - origem       object
 13  UF - destino             object
 14  Município - destino      object
 15  Situação Cadastral       object
dtypes: int64(1), object(15)
memory usage: 381.6+ MB


In [ ]:
Dados_IBAMA_2013a2023.head(5)

In [60]:
# Removendo todos outros modais, preservando apenas o rodoviário.

Dados_IBAMA_teste = Dados_IBAMA_2013a2023.query("`Tipo de Transporte` == 'Rodoviário'")

In [61]:
# Removendo zeros e valores nulos.

Dados_IBAMA_teste = Dados_IBAMA_teste[(Dados_IBAMA_teste != 0).all(axis=1)].dropna()
(Dados_IBAMA_teste == 0).sum()

,0
CNPJ,0
Razão Social,0
Estado,0
Município,0
Ano,0
Produto,0
Quantidade Transportada,0
Unidade de Medida,0
Tipo de Transporte,0
Tipo de Armazenamento,0


In [62]:
# Apenas unidades de medidas de kg e litros

Dados_IBAMA_teste = Dados_IBAMA_teste.query(
    "`Unidade de Medida` in ['kilogramas', 'Litro', 'Metro Cúbico',  'Tonelada']"
)

In [30]:
print(Dados_IBAMA_teste['Unidade de Medida'].unique())

['Tonelada' 'kilogramas' 'Litro' 'Metro Cúbico' 'Grama' 'Miligrama'
 'Militro']


In [31]:
# 🔸 Lista de palavras-chave para filtrar produtos perigosos ou especiais
# Inclui:
# - Combustíveis e derivados (ex: gasolina, diesel, etanol, GLP, querosene, óleo, lubrificantes)
# - Substâncias químicas perigosas (ex: ácidos, nitratos, solventes, cloro, amônia, formaldeído, etc.)
# - Termos relacionados a risco (inflamável, tóxico, corrosivo, oxidante, radioativo, explosivo...)
# - Resíduos e efluentes industriais (lama, lodo, borra, emulsão, catalisadores, defensivos, fertilizantes químicos)
# - Gases e criogênicos (oxigênio, nitrogênio, CO2, gás comprimido, criogênico...)
# - Metais pesados e compostos correlatos (mercúrio, chumbo, dicromato, etc.)
# - Produtos industriais diversos (tintas, vernizes, betume, adesivos químicos, aditivos, borracha sintética...)

palavras_perigosas = [
    'gasolina', 'diesel', 'etanol', 'álcool', 'biodiesel', 'combustível',
    'querosene', 'gnv', 'gás liquefeito', 'gás natural', 'butano', 'propano',
    'glp', 'naphta', 'nafta', 'óleo', 'lubrificante',

    'ácido', 'base', 'alcalino', 'hidróxido', 'nitrato', 'peróxido',
    'hipoclorito', 'cloro', 'amônia', 'soda cáustica', 'cloreto', 'sulfato',
    'fosfato', 'ácetico', 'acético', 'cianeto', 'anidro', 'solvente',
    'tolueno', 'xileno', 'benzeno', 'acetona', 'formaldeído', 'fenol',
    'butil', 'metanol', 'isopropanol', 'hexano', 'heptano', 'éter',

    'inflamável', 'tóxico', 'corrosivo', 'explosivo', 'oxidante',
    'radioativo', 'perigoso', 'reativo', 'irritante', 'comburente',
    'poluente', 'resíduo perigoso', 'resíduo químico',

    'resíduo', 'lama', 'lodo', 'efluente', 'emulsão', 'borra', 'bagaço químico',
    'catalisador', 'pesticida', 'herbicida', 'inseticida', 'defensivo',
    'fertilizante químico', 'corante industrial', 'pigmento químico',

    'oxigênio', 'nitrogênio', 'hélio', 'argônio', 'co2', 'co', 'freon',
    'refrigerante', 'criogenico', 'criogênico', 'gás comprimido', 'gás liquefeito',

    'mercúrio', 'chumbo', 'cádmio', 'arsênio', 'cobre', 'níquel',
    'zinco', 'manganês', 'crômico', 'cromato', 'dicromato',

    'betume', 'asfalto', 'tinta', 'verniz', 'cola industrial',
    'adesivo químico', 'detergente industrial', 'espessante químico',
    'aditivo químico', 'plástico fundido', 'borracha sintética'
]

mask = Dados_IBAMA_teste['Produto'].str.lower().str.contains('|'.join(palavras_perigosas), na=False)
Dados_IBAMA_teste = Dados_IBAMA_teste[mask]


In [ ]:
contagem_valores = Dados_IBAMA_teste['Produto'].value_counts()
print(contagem_valores)

In [ ]:
Dados_IBAMA_teste['Produto'].value_counts().tail(50)

In [69]:
# Inicializa colunas novas
Dados_IBAMA_teste['qntd_Litros'] = 0.0
Dados_IBAMA_teste['qntd_Kg'] = 0.0

def normalizar(row):
    valor = row['Quantidade Transportada']
    unidade = str(row['Unidade de Medida']).strip()  # mantém maiúsculas/minúsculas conforme estão

    # tenta converter para float
    try:
        valor = float(str(valor).replace(',', '.'))
    except:
        valor = 0.0

    # --- CONVERSÃO PARA LITRO ---
    if unidade in ['Metro Cúbico']:
        row['Quantidade Transportada'] = valor * 1000
        row['Unidade de Medida'] = 'Litro'

    # --- CONVERSÃO PARA KILOGRAMA ---
    elif unidade in ['Tonelada']:
        row['Unidade de Medida'] = 'kilogramas'

    return row

# Aplica a normalização
Dados_IBAMA_teste = Dados_IBAMA_teste.apply(normalizar, axis=1)

# Preenche as colunas finais
Dados_IBAMA_teste.loc[Dados_IBAMA_teste['Unidade de Medida'] == 'Litro', 'qntd_Litros'] = Dados_IBAMA_teste['Quantidade Transportada']
Dados_IBAMA_teste.loc[Dados_IBAMA_teste['Unidade de Medida'] == 'kilogramas', 'qntd_Kg'] = Dados_IBAMA_teste['Quantidade Transportada']

# Remove as colunas antigas
Dados_IBAMA_teste = Dados_IBAMA_teste.drop(columns=['Quantidade Transportada', 'Unidade de Medida'])

# Visualiza resultado
print(Dados_IBAMA_teste[['qntd_Litros','qntd_Kg']].head(10))


KeyError: 'Quantidade Transportada'

In [68]:
Dados_IBAMA_teste.head(20)

,CNPJ,Razão Social,Estado,Município,Ano,Produto,Tipo de Transporte,Tipo de Armazenamento,Plano de Emergência,UF - origem,Município - origem,UF - destino,Município - destino,Situação Cadastral,qntd_Litros,qntd_Kg
200354,08.895.599/0001-80,COOPERCELL COOPERATIVA DE TRANSPORTES DE CARGA...,SAO PAULO,RIBEIRAO PIRES,2013,Hidróxido de sódio (soda cáustica),Rodoviário,Tanques,Sim,SAO PAULO,GUARUJA,SAO PAULO,GUARULHOS,Ativa,0.0,"30.951,00"
200353,08.895.599/0001-80,COOPERCELL COOPERATIVA DE TRANSPORTES DE CARGA...,SAO PAULO,RIBEIRAO PIRES,2013,Hidróxido de sódio (soda cáustica),Rodoviário,Tanques,Sim,SAO PAULO,GUARUJA,SAO PAULO,GUARULHOS,Ativa,0.0,"25.978,00"
200352,08.895.599/0001-80,COOPERCELL COOPERATIVA DE TRANSPORTES DE CARGA...,SAO PAULO,RIBEIRAO PIRES,2013,Hidróxido de sódio (soda cáustica),Rodoviário,Tanques,Sim,SAO PAULO,COTIA,MINAS GERAIS,BELO HORIZONTE,Ativa,0.0,"13.190,00"
200288,08.895.599/0001-80,COOPERCELL COOPERATIVA DE TRANSPORTES DE CARGA...,SAO PAULO,RIBEIRAO PIRES,2013,Hidróxido de sódio (soda cáustica),Rodoviário,Tanques,Sim,SAO PAULO,COTIA,RIO DE JANEIRO,RESENDE,Ativa,0.0,"34.830,00"
200269,08.895.599/0001-80,COOPERCELL COOPERATIVA DE TRANSPORTES DE CARGA...,SAO PAULO,RIBEIRAO PIRES,2013,Hidróxido de sódio (soda cáustica),Rodoviário,Tanques,Sim,SAO PAULO,COTIA,MINAS GERAIS,UBERLANDIA,Ativa,0.0,"14.139,00"
200225,08.895.599/0001-80,COOPERCELL COOPERATIVA DE TRANSPORTES DE CARGA...,SAO PAULO,RIBEIRAO PIRES,2013,Hidróxido de sódio (soda cáustica),Rodoviário,Tanques,Sim,SAO PAULO,COTIA,MINAS GERAIS,SAO GONCALO DO SAPUCAI,Ativa,0.0,"13.840,00"
200121,08.895.599/0001-80,COOPERCELL COOPERATIVA DE TRANSPORTES DE CARGA...,SAO PAULO,RIBEIRAO PIRES,2013,Hidróxido de sódio (soda cáustica),Rodoviário,Tanques,Sim,SAO PAULO,GUARUJA,SAO PAULO,COTIA,Ativa,0.0,"31.222,00"
198898,08.895.599/0001-80,COOPERCELL COOPERATIVA DE TRANSPORTES DE CARGA...,SAO PAULO,RIBEIRAO PIRES,2013,Hidróxido de sódio (soda cáustica),Rodoviário,Tanques,Sim,SAO PAULO,GUARULHOS,MINAS GERAIS,UBERABA,Ativa,0.0,"25.949,00"
198535,08.895.599/0001-80,COOPERCELL COOPERATIVA DE TRANSPORTES DE CARGA...,SAO PAULO,RIBEIRAO PIRES,2013,Hidróxido de sódio (soda cáustica),Rodoviário,Tanques,Sim,SAO PAULO,SUZANO,SAO PAULO,LIMEIRA,Ativa,0.0,"13.960,00"
198498,08.895.599/0001-80,COOPERCELL COOPERATIVA DE TRANSPORTES DE CARGA...,SAO PAULO,RIBEIRAO PIRES,2013,Hipocloritos de sódio (exceto água sanitária),Rodoviário,Tanques,Sim,SAO PAULO,CUBATAO,SAO PAULO,SAO PAULO,Ativa,0.0,"7.170,00"
